<center> <h1>EnerjiSA Datathon</h1> </center>
<!-- <img src="https://upload.wikimedia.org/wikipedia/commons/b/b8/Photovoltaik_Dachanlage_Hannover_-_Schwarze_Heide_-_1_MW.jpg" style="margin: 0 auto; width: 100%; max-width: 950px; max-height: 300px" frameborder="0" scrolling="auto"/> -->

#### Previous notebook: [EnerjiSA Time Series Analysis Starter](https://www.kaggle.com/sarperyurttas/enerjisa-time-series-analysis-starter)

<a id="Import Libraries"></a>
<center> <h1>Import Libraries</h1> </center>

In [1]:
# !pip install flaml
# !pip install lightgbm
# !pip install flaml

In [2]:
from sklearn.metrics import mean_squared_error
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
# from flaml import AutoML
import optuna
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.float_format', lambda x: '%.8f' % x)

<a id="Data Preprocessing"></a>
<center> <h1>Data Preprocessing</h1> </center>


In [3]:
# read csv files
gen = pd.read_csv('generation.csv', delimiter=';', decimal=',')
temp = pd.read_csv('temperature.csv', delimiter=';', decimal=',')

In [4]:
# slicing frames due to deficiencies in data
df_train = pd.concat([gen.iloc[:25560], temp.iloc[:25560].drop('DateTime', axis=1)], axis=1)

In [5]:
def preprocess(df):
    df['DateTime'] = pd.to_datetime(df['DateTime'])
    df = df.set_index('DateTime')
    df['WWCode'] = df['WWCode'].fillna(0).astype(int)
    # Weather codes filled with "0" because this category means "Cloud development not observed or not observable" reference: https://www.jodc.go.jp/data_format/weather-code.html
    return df

In [6]:
df_test = temp.iloc[25560:26304] # dates between 2021-12-1 and 2021-12-31
df_train = preprocess(df_train)
df_test = preprocess(df_test)

In [7]:
# category 84 samples changed to 83 because category 84 is not found in train set and 83 is the nearest category to 84
df_test.loc[df_test['WWCode'] == 84, 'WWCode'] = 83 

<a id="Preparing Data For Modelling"></a>
<center> <h1>Preparing Data For Modelling</h1> </center>


In [8]:
def create_features(df, label=None):
    """
    Creates time series features from datetime index
    """
    df_copy = df.copy()
    df_copy['date'] = df_copy.index
    df_copy['hour'] = df_copy['date'].dt.hour
    df_copy['dayofweek'] = df_copy['date'].dt.dayofweek
    df_copy['quarter'] = df_copy['date'].dt.quarter
    df_copy['month'] = df_copy['date'].dt.month
    df_copy['year'] = df_copy['date'].dt.year
    df_copy['dayofyear'] = df_copy['date'].dt.dayofyear
    df_copy['dayofmonth'] = df_copy['date'].dt.day
    df_copy['weekofyear'] = df_copy['date'].dt.weekofyear
    
    X = df_copy[['hour','dayofweek','quarter','month','year',
           'dayofyear','dayofmonth','weekofyear']]
    
    if label:
        y = df_copy[label]
        X = pd.concat([df.drop(label, axis=1), X], axis=1)
        return X, y
    else:
        X = pd.concat([df, X], axis=1)
        return X

In [9]:
def split_train(df, split_date):
    train = df.loc[df.index <= split_date].copy()
    val = df.loc[df.index > split_date].copy()
    return train, val

In [10]:
split_date = '2021-05-01'
df_train, df_val = split_train(df_train, split_date)

In [11]:
x_train, y_train = create_features(df_train, label='Generation')
x_val, y_val = create_features(df_val, label='Generation')
x_test = create_features(df_test)

In [26]:
x_train

,AirTemperature,ComfortTemperature,RelativeHumidity,WindSpeed,WindDirection,WWCode,EffectiveCloudCover,hour,dayofweek,quarter,month,year,dayofyear,dayofmonth,weekofyear
DateTime,,,,,,,,,,,,,,,
2019-01-01 00:00:00,-1.70000000,-6.10000000,75.30000000,3.60000000,60.00000000,10,6.30000000,0,1,1,1,2019,1,1,1
2019-01-01 01:00:00,-1.80000000,-5.30000000,75.30000000,2.60000000,70.00000000,10,4.50000000,1,1,1,1,2019,1,1,1
2019-01-01 02:00:00,-2.00000000,-6.00000000,74.70000000,3.10000000,80.00000000,10,5.50000000,2,1,1,1,2019,1,1,1
2019-01-01 03:00:00,-1.90000000,-5.90000000,76.40000000,3.10000000,60.00000000,10,7.60000000,3,1,1,1,2019,1,1,1
2019-01-01 04:00:00,-2.00000000,-7.10000000,76.40000000,4.60000000,60.00000000,10,6.50000000,4,1,1,1,2019,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-04-30 20:00:00,22.60000000,20.70000000,20.00000000,1.00000000,30.00000000,3,3.00000000,20,4,2,4,2021,120,30,17
2021-04-30 21:00:00,21.70000000,20.10000000,23.10000000,1.50000000,20.00000000,4,4.60000000,21,4,2,4,2021,120,30,17
2021-04-30 22:00:00,20.60000000,19.20000000,25.80000000,0.50000000,20.00000000,3,2.50000000,22,4,2,4,2021,120,30,17


<a id="Hyperparameter Tuning"></a>
<center> <h1>Hyperparameter Tuning</h1> </center>


In [12]:
datasets = {'x_train': x_train,
            'y_train': y_train,
            'x_val': x_val,
            'y_val': y_val,
            'x_test': x_test
            }

In [13]:
def pred_model(model, **datasets):
    pred_val = model.predict(datasets['x_val'])
    pred_val = pd.DataFrame(pred_val, index=datasets['x_val'].index, columns=['pred'])
    
    pred_test = model.predict(datasets['x_test'])

    return pred_test, pred_val

<a id="FLAML"></a>
<center> <h1>FLAML</h1> </center>


In [14]:
# automl = AutoML()
# settings = {
#     "time_budget": 300,  
#     "metric": 'rmse',
#     "estimator_list": ['lgbm'],
#     "task": 'regression',
# }
# automl.fit(X_train=datasets['x_train'], y_train=datasets['y_train'], **settings)

In [15]:
# model = LGBMRegressor(**automl.best_config)    
# model.fit(datasets['x_train'], datasets['y_train'])

In [16]:
# pred_flaml, pred_val_flaml = pred_model(model, **datasets)

<a id="OPTUNA"></a>
<center> <h1>OPTUNA</h1> </center>


In [17]:
def objective(trial):
    learning_rate = trial.suggest_float("learning_rate", 1e-2, 0.25, log=True)
    reg_lambda = trial.suggest_loguniform("reg_lambda", 1e-8, 100.0)
    reg_alpha = trial.suggest_loguniform("reg_alpha", 1e-8, 100.0)
    subsample = trial.suggest_float("subsample", 0.1, 1.0)
    colsample_bytree = trial.suggest_float("colsample_bytree", 0.1, 1.0)
    max_depth = trial.suggest_int("max_depth", 1, 7)
    early_stopping_rounds = trial.suggest_int("early_stopping_rounds", 100, 500)
    n_estimators = trial.suggest_int("n_estimators", 0, 10000)
    
    model = XGBRegressor(
        random_state=50,
        tree_method="gpu_hist",
        gpu_id=0,
        predictor="gpu_predictor",
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        reg_lambda=reg_lambda,
        reg_alpha=reg_alpha,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        max_depth=max_depth,
    )
    model.fit(
        datasets['x_train'],
        datasets['y_train'],
        early_stopping_rounds=early_stopping_rounds,
        eval_set=[(datasets['x_val'], datasets['y_val'])],
        verbose=1200,
    )
    preds_valid = model.predict(datasets['x_val'])
    rmse = mean_squared_error(datasets['y_val'], preds_valid, squared=False)

    return rmse

In [18]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=20)

[I 2022-02-28 15:42:54,334] A new study created in memory with name: no-name-5eef5e8e-cb30-4be1-a8fa-3be4a3a0533a


[0]	validation_0-rmse:147.46828
[472]	validation_0-rmse:30.89779


[I 2022-02-28 15:43:04,555] Trial 0 finished with value: 24.217572818263186 and parameters: {'learning_rate': 0.22587671844591384, 'reg_lambda': 0.017479591267211245, 'reg_alpha': 1.732781477562603e-08, 'subsample': 0.2090063729622258, 'colsample_bytree': 0.9351726599072918, 'max_depth': 7, 'early_stopping_rounds': 450, 'n_estimators': 6536}. Best is trial 0 with value: 24.217572818263186.


[0]	validation_0-rmse:160.01637
[759]	validation_0-rmse:24.31786


[I 2022-02-28 15:43:10,696] Trial 1 finished with value: 22.965385793343703 and parameters: {'learning_rate': 0.15193594102027522, 'reg_lambda': 9.121957141067638e-08, 'reg_alpha': 1.6643812772351702e-07, 'subsample': 0.6855411571737606, 'colsample_bytree': 0.8064099912277553, 'max_depth': 5, 'early_stopping_rounds': 489, 'n_estimators': 2770}. Best is trial 1 with value: 22.965385793343703.


...

[I 2022-02-28 15:48:31,431] Trial 17 finished with value: 23.175024554293515 and parameters: {'learning_rate': 0.01860311812150148, 'reg_lambda': 0.08801344464861816, 'reg_alpha': 0.0223530664103807, 'subsample': 0.5041628757747523, 'colsample_bytree': 0.7740378411787596, 'max_depth': 3, 'early_stopping_rounds': 365, 'n_estimators': 6367}. Best is trial 13 with value: 22.673224188000535.


[0]	validation_0-rmse:181.42940
[1200]	validation_0-rmse:40.09516
[2400]	validation_0-rmse:34.42894
[3600]	validation_0-rmse:31.70663
[4800]	validation_0-rmse:29.94456
[4966]	validation_0-rmse:29.75515


[I 2022-02-28 15:48:48,979] Trial 18 finished with value: 29.75514350344508 and parameters: {'learning_rate': 0.016450202765062327, 'reg_lambda': 9.167864732336193, 'reg_alpha': 0.0009694360670785971, 'subsample': 0.8660994021575108, 'colsample_bytree': 0.6402085836740057, 'max_depth': 2, 'early_stopping_rounds': 218, 'n_estimators': 4967}. Best is trial 13 with value: 22.673224188000535.


[0]	validation_0-rmse:180.99124
[1200]	validation_0-rmse:25.77754
[2400]	validation_0-rmse:23.92089
[3600]	validation_0-rmse:23.36092
[4046]	validation_0-rmse:23.39931


[I 2022-02-28 15:49:05,391] Trial 19 finished with value: 23.33383953217145 and parameters: {'learning_rate': 0.03282882081481022, 'reg_lambda': 0.0032457965780593096, 'reg_alpha': 0.00034031746285960175, 'subsample': 0.8607609968151619, 'colsample_bytree': 0.4723728931705984, 'max_depth': 3, 'early_stopping_rounds': 339, 'n_estimators': 8596}. Best is trial 13 with value: 22.673224188000535.


In [19]:
optuna_params = study.best_params
optuna_params

{'learning_rate': 0.011415367681056826,
 'reg_lambda': 0.8078672295819468,
 'reg_alpha': 0.0026055083430921344,
 'subsample': 0.8091275924233325,
 'colsample_bytree': 0.7199608834465959,
 'max_depth': 3,
 'early_stopping_rounds': 371,
 'n_estimators': 9904}

In [20]:
model = XGBRegressor(
    random_state=50,
    tree_method="gpu_hist",
    gpu_id=0,
    predictor="gpu_predictor",
    **optuna_params
)
model.fit(
    datasets['x_train'],
    datasets['y_train'],
    early_stopping_rounds=optuna_params['early_stopping_rounds'],
    eval_set=[(datasets['x_val'], datasets['y_val'])],
    verbose=1000,
)

[15:49:05] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576: 
Parameters: { "early_stopping_rounds" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-rmse:181.02789
[1000]	validation_0-rmse:29.66521
[2000]	validation_0-rmse:25.59036
[3000]	validation_0-rmse:24.11302
[4000]	validation_0-rmse:23.48376
[5000]	validation_0-rmse:23.16133
[6000]	validation_0-rmse:22.96908
[7000]	validation_0-rmse:22.80327
[8000]	validation_0-rmse:22.72942
[9000]	validation_0-rmse:22.69370
[9219]	validation_0-rmse:22.67560


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.7199608834465959,
             early_stopping_rounds=371, enable_categorical=False, gamma=0,
             gpu_id=0, importance_type=None, interaction_constraints='',
             learning_rate=0.011415367681056826, max_delta_step=0, max_depth=3,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=9904, n_jobs=8, num_parallel_tree=1,
             predictor='gpu_predictor', random_state=50,
             reg_alpha=0.0026055083430921344, reg_lambda=0.8078672295819468,
             scale_pos_weight=1, subsample=0.8091275924233325,
             tree_method='gpu_hist', validate_parameters=1, verbosity=None)

In [21]:
pred_optuna, pred_val_optuna = pred_model(model, **datasets)

#### Simple trick: Negative predicted and night values are clipped to 0

In [22]:
def clip_nights(pred, test = False):
    nights = [21, 22 , 23 , 0, 1, 2, 3, 4]
    if not test:
        for time, row in pred.iterrows():
            if time.hour in nights or row['pred'] < 0:
                row['pred'] = 0
        return pred
    else:
        for i, hour in enumerate(pd.to_datetime(pred['DateTime'])):
            if hour.hour in nights or pred.loc[i, 'Generation'] < 0:
                pred.loc[i, 'Generation'] = 0
        return pred

In [23]:
# pred_val_flaml = clip_nights(pred_val_flaml)
pred_val_optuna = clip_nights(pred_val_optuna)

In [24]:
# rmse_flaml = mean_squared_error(datasets['y_val'], pred_val_flaml, squared=False)
rmse_optuna = mean_squared_error(datasets['y_val'], pred_val_optuna, squared=False)
# print('Flaml RMSE: ', rmse_flaml)
print('Optuna RMSE: ', rmse_optuna)

Optuna RMSE:  22.063721892469616


In [27]:
pred_val_optuna

,pred
DateTime,
2021-05-01 01:00:00,0.00000000
2021-05-01 02:00:00,0.00000000
2021-05-01 03:00:00,0.00000000
2021-05-01 04:00:00,0.00000000
2021-05-01 05:00:00,5.74300146
...,...
2021-11-30 19:00:00,7.65563250
2021-11-30 20:00:00,8.55105495
2021-11-30 21:00:00,0.00000000


#### Comparing predictions

In [25]:
plt.figure(figsize=(50,10))
plt.plot(pred_val_flaml)
plt.plot(pred_val_optuna)
plt.plot(y_val)
plt.legend(['Flaml','Optuna', 'Desired'])
plt.show()

NameError: name 'pred_val_flaml' is not defined

<Figure size 3600x720 with 0 Axes>

In [ ]:
submission = pd.read_csv('../input/enerjisa-enerji-veri-maratonu/sample_submission.csv')
submission['Generation'] = pred_flaml

In [ ]:
submission2 = pd.read_csv('../input/enerjisa-enerji-veri-maratonu/sample_submission.csv')
submission2['Generation'] = pred_optuna

In [ ]:
submission = clip_nights(submission, test=True)
submission2 = clip_nights(submission2, test=True)

In [ ]:
plt.figure(figsize=(31,10))
plt.plot(submission['Generation'])
plt.plot(submission2['Generation'])
plt.show()

In [ ]:
submission.to_csv('submission.csv', index=False)
submission2.to_csv('submission2.csv', index=False)